# main.py — converted to notebook

This notebook contains the contents of `main.py` from the project. 
Run the code cells below in a Python 3 kernel.

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from IPython.display import Image
from tool import send_email, received_emails

load_dotenv()

llm_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
email_tools = [send_email, received_emails]
llm_with_tools = llm_model.bind_tools(tools=email_tools)


class EmailClassification(BaseModel):
    classify: str
    status: str | None = None
    context: dict | None = None


def classify_email_request(user_request: str):
    prompt = f"""
    You are an intelligent email classification assistant.
    Your task is to analyze the user request and classify it for email operations.

    Follow these rules:
    1. Determine if the request is related to emails.
       - If unrelated, respond with:
         {
             "classify": "NOT_EMAIL",
             "status": null,
             "context": null
         }

    2. If the request is related to emails, classify the operation type:
       - "EMAIL_SEND": User wants to send an email.
       - "EMAILS_CHECK": User wants to retrieve or read emails.

    3. For "EMAILS_CHECK", identify the filter/status:
       - Valid filters: ALL, SEEN, UNSEEN, ANSWERED, UNANSWERED, FLAGGED, UNFLAGGED,
         DRAFT, DELETED, NEW, OLD, RECENT,
         FROM "email@example.com", TO "email@example.com", SUBJECT "keyword",
         BODY "keyword", SINCE DD-MMM-YYYY, BEFORE DD-MMM-YYYY
       - Default to UNSEEN if not specified.

    4. For "EMAIL_SEND", extract context if available:
       - "to": recipient email address
       - "subject": email subject
       - "content": email body
       - Example:
         "context": {
             "to": "example@example.com",
             "subject": "Meeting Reminder",
             "content": "Don't forget our meeting at 10 AM."
         }

    5. Only output structured JSON as per the EmailClassification model.
       Do not include any extra text.

    User request: "{user_request}"
    """

    structured_llm = llm_model.with_structured_output(EmailClassification)
    classification_result = structured_llm.invoke(prompt)
    print(classify_email_request)
    return {"classification": classification_result}


def check_email(classification: dict):
    cls = classification.get("classification").classify
    if cls == "EMAIL_SEND":
        return "generate"
    elif cls == "EMAILS_CHECK":
        return "fetch"
    else:
        return None


def generate(classification: dict):
    pass


def fetch(classification: dict):
    pass


email_state_graph = StateGraph(dict)

email_state_graph.add_node("classify_email_request", classify_email_request)
email_state_graph.add_node("generate", generate)
email_state_graph.add_node("fetch", fetch)

email_state_graph.add_edge(START, "classify_email_request")

email_state_graph.add_conditional_edges(
    "classify_email_request",
    check_email,
    {"generate": "generate", "fetch": "fetch"}
)

email_state_graph.add_edge("generate", END)
email_state_graph.add_edge("fetch", END)

chat_graph = email_state_graph.compile()
chat_graph.invoke({"user_request":"send a mail to akshaykalangi54@gmail.com happy birthday"})

ValueError: Invalid format specifier ' "NOT_EMAIL",
             "status": null,
             "context": null
         ' for object of type 'str'